# To-Do:

- InputStandardize vs Normalize

- check hyperparam/prior definitions (scaled space vs raw space)

- keep in mind modality of NaN results in emittance

- Try different number of steps along measurement dimension

- inrease dimensionality of tuning space

- fix legend location (only plot on 1 heatmap)

# In this notebook, we fit a gpytorch GP to a simple emittance model with 1 tuning parameter. We use the GP to evaluate the Expected Information Gain toward the result of a grid-scan minimization routine.

In [1]:
import torch
from emitutils import toy_beam_size_squared_nd, fit_gp_model_emittance
from utils import unif_random_sample_domain
from matplotlib import pyplot as plt
from algorithms import GridMinimizeEmittance
from acquisition import ExpectedInformationGain
from botorch.optim import optimize_acqf
import time
from mpl_toolkits.axes_grid1 import make_axes_locatable
import copy

# Settings

In [2]:
# domain = torch.tensor([[-2,2], [-65,35]]).double() #the acquisition domain, must have shape = (ndim, 2)
domain = torch.tensor([[-3,1], [-3,1], [-40,60]]).double() #the acquisition domain, must have shape = (ndim, 2)
ndim = domain.shape[0]                               #where domain[i,0] and domain[i,1] represent
                                                        #the lower and upper bounds of the ith input dimension
                                                        #(these same bounds will be applied to the sampled execution paths) 

    

    
n_samples = 100 #number of posterior samples on which to evaluate execution paths
n_steps_tuning_params = 11 #number of steps per dimension in the posterior sample grid scans 
n_steps_measurement_param = 11
squared = True #whether or not to minimize the "emittance squared" (which can be negative according to the model)





random_acq = True
n_trials = 10
n_iter = 50
n_obs_init = 5 #number of random observations on which to initialize model


In [3]:
# domain = torch.tensor([[-2,2], [-2,2], [-65,35]]).double() #the acquisition domain, must have shape = (ndim, 2)
#                                                         #where domain[i,0] and domain[i,1] represent
#                                                         #the lower and upper bounds of the ith input dimension
#                                                         #(these same bounds will be applied to the sampled execution paths)
        
# ndim = domain.shape[0]
    





# n_samples = 100 #number of posterior samples on which to evaluate execution paths
# n_steps_tuning_params = 11 #number of steps per dimension in the posterior sample grid scans 
# n_steps_measurement_param = 11
# squared = True #whether or not to minimize the "emittance squared" (which can be negative according to the model)





# random_acq = True
# n_trials = 20
# n_iter = 20
# n_obs_init = 5 #number of random observations on which to initialize model

# Initialize

In [4]:
trial_data = {}
trial_data['settings'] = {'domain':domain,
                         'ndim':ndim,
                         'n_obs_init': n_obs_init,
                         'n_samples':n_samples,
                         'n_steps_tuning_params':n_steps_tuning_params,
                         'n_steps_measurement_param': n_steps_measurement_param,
                         'n_trials':n_trials,
                         'n_iter':n_iter,
                         'squared':squared,
                         'random_acq':random_acq}

for trial in range(n_trials):
    torch.manual_seed(trial)

    #build ndim dimensional parabolic target function
    target_func = toy_beam_size_squared_nd


    ##########################################
    #Observe target function n_obs_init times using a uniform sample of the domain
    x_obs = unif_random_sample_domain(n_samples = n_obs_init, domain = domain)
    y_obs = target_func(x_obs) 




    #fit model on initial observations
    model = fit_gp_model_emittance(x_obs, y_obs*1.e6)

    algo = GridMinimizeEmittance(domain = domain, 
                   n_samples = n_samples, 
                   n_steps_tuning_params = n_steps_tuning_params,
                    n_steps_measurement_param = n_steps_measurement_param,
                    squared = squared)
    
    rng_state = torch.get_rng_state()
    
    acq_fn = ExpectedInformationGain(model = model, algo = algo)

    if random_acq:
        x_next = None
    else:
        x_next, _ = optimize_acqf(
            acq_function=acq_fn,
            bounds=acq_fn.algo.domain.T,
            q=1,
            num_restarts=20,
            raw_samples=100,
            options={},
            )
    
    iter_data = {}
    iter_data[0] = {'x_obs': x_obs,
                   'y_obs': y_obs,
                    'x_next': x_next,
                   'model':  copy.deepcopy(model),
                   'rng_state': rng_state}
    
    for i in range(1, n_iter+1):
        start = time.time()
        print('Iteration', trial*n_iter + i, '/', n_trials*n_iter)
        
        if random_acq:
            x_new = unif_random_sample_domain(n_samples = 1, domain = domain)
        else:
            x_new = x_next
            
        y_new = target_func(x_new)

        x_obs = torch.cat((x_obs, x_new), dim=0)
        y_obs = torch.cat((y_obs, y_new), dim=0)

        model = fit_gp_model_emittance(x_obs, y_obs*1.e6)

        rng_state = torch.get_rng_state()
        
        acq_fn = ExpectedInformationGain(model = model, algo = algo)

        if random_acq:
            x_next = None
        else:
            x_next, _ = optimize_acqf(
                acq_function=acq_fn,
                bounds=acq_fn.algo.domain.T,
                q=1,
                num_restarts=20,
                raw_samples=100,
                options={},
                )
            
        end = time.time()
        print('Operation took', end - start, 'seconds.')
        
        iter_data[i] = {'x_obs': x_obs,
                   'y_obs': y_obs,
                    'x_next': x_next,
                   'model':  copy.deepcopy(model),
                   'rng_state': rng_state}

    trial_data[trial] = iter_data 


C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


Iteration 1 / 500


C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


Operation took 0.30295801162719727 seconds.
Iteration 2 / 500


C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  warnings.warn(


...

C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  warnings.warn(
C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(


Operation took 0.27640461921691895 seconds.
Iteration 499 / 500


C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  warnings.warn(
C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(


Operation took 0.29382920265197754 seconds.
Iteration 500 / 500
Operation took 0.30828237533569336 seconds.


C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  warnings.warn(
C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(


In [5]:
# import dill
# with open('MC-Emittance-Phys-Random-2d-Results.pkl', 'wb') as f:
#     dill.dump(trial_data, f)

In [6]:
import dill
with open('MC-Emittance-NonPhys-Random-3d-Results-test.pkl', 'wb') as f:
    dill.dump(trial_data, f)